### Data Set 1: cano.txt (Arthur Conan Doyle's entire Sherlock Holmes works. The canon includes all the stories and novels listed on this page in a single document.) 


*   Remove punctuation Marks.
*   Convert Upper case to lower case.
*   Remove Tabs.
*   Remove special characters 


Our goal is to normalize data before generating key value pairs so that we can have the 
actual occurrence of each word. 


In [ ]:
!cat mapper.py

# -*- coding: utf-8 -*-
"""mapper.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1yCwGyMXJT2qt3_58aLOOiJXO0GIaPcJd
"""



import sys
import io
import re
import nltk
nltk.download('stopwords',quiet=True)
from nltk.corpus import stopwords
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

stop_words = set(stopwords.words('english'))
input_stream = io.TextIOWrapper(sys.stdin.buffer, encoding='latin1')
for line in input_stream:
  line = line.strip()
  line = re.sub(r'[^\w\s]', '',line)
  line = line.lower()
  for x in line:
    if x in punctuations:
      line=line.replace(x, " ") 

  words=line.split()
  for word in words: 
    if word not in stop_words:
      print('%s\t%s' % (word, 1))

In [ ]:
!cat reducer.py

# -*- coding: utf-8 -*-
"""reducer.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1YzJ-vUsO5VYCyMrfPMow3s2IdxXkyQ0i
"""

from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    line=line.lower()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)
    try:
      count = int(count)
    except ValueError:
      #count was not a number, so silently
      #ignore/discard this line
      continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print ('%s\t%s' % (current_word, current_count))
        current_count 

In [ ]:
!cat cano.txt | python mapper.py | sort -k1,1 | python reducer.py

Streaming output truncated to the last 5000 lines.
seveneighths	1
sevenmile	1
sevenpercent	1
sevenshilling	1
seventeen	6
seventeenth	4
seventh	6
seventhirty	4
seventy	5
seventyfive	1
seventyfour	1
sever	1
several	180
severe	20
severed	3
severely	8
severity	3
severn	1
seville	1
sewingmachine	1
sewn	1
sex	6
seymour	1
sh	1
shabbily	1
shabby	3
shabbygenteel	1
shack	1
shade	16
shaded	3
shades	2
shading	3
shadow	82
shadowed	12
shadowmisery	1
shadows	21
shadowy	8
shadwell	1
shady	1
shaft	5
shafter	8
shafters	8
shaftesbury	1
shafts	2
shag	6
shaggy	3
shahgunge	1
shake	15
shakedown	1
shaken	40
shakespeare	1
shaking	39
shaky	5
shall	652
shallow	3
sham	2
shambles	1
shambling	2
shame	8
shamefaced	3
shameful	2
shamefully	4
shamefullyshamefully	1
shameless	1
shant	3
shanty	1
shape	36
shapea	1
shaped	5
shapeless	5
shapely	1
shapes	4
shaping	2
shard	1
shards	2
share	49
shared	19
sharer	1
shares	4
sharing	8
shark	5
sharks	1
sharp	93
sharpen	2
sharpened	3
sharpens	1
sharper	1
sharpers	1
sharpest	1
sharpe

### Data Set 2: US_AirlLines.txt
Take a positive number N>0 and find N top trending words in the corpus. Data set for the 
task is Data set 2 (already normalized)

In [ ]:
!cat mapper#2.py

#!/usr/bin/python
# import sys because we need to read and write data to STDIN and STDOUT
import sys
import re
# reading entire line from STDIN (standard input)
for line in sys.stdin:
# to remove leading and trailing whitespace
	line = line.strip()
	line = re.sub(r'[^\w\s]', '', line)
	line = line.lower()
# split the line into words
	words = line.split()
# we are looping over the words array and printing the word
# with the count of 1 to the STDOUT
	for word in words:
# write the results to STDOUT (standard output);
# what we output here will be the input for the
# Reduce step, i.e. the input for reducer.py
		print('%s\t%s' % (word, 1))



In [ ]:
!cat reducer#2.py

#!/usr/bin/env python
"""reducer.py"""
from heapq import nlargest
from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None
N = 20
word_dict={}

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            word_dict[current_word]=current_count
        current_count = count
        current_word = word

# do not forget to output the last word if nee

In [ ]:
!cat US_AirlLines.txt | python mapper#2.py | sort -k1,1 | python reducer#2.py

flight	4667
unit	4020
usairway	3001
americanair	2960
southwestair	2459
jetblu	2189
get	1619
thank	1326
tco	1211
http	1153
can	1116
cancel	1064
now	1025
help	993
hour	977
just	965
time	917
custom	854
delay	849
will	784


### Data Set 3: Literacy_and_Literacy_Rate.csv
Use data set 3 to perform following tasks in distributed environment  
1. Count total number of males’ literates and females literates. 
2. Find average literacy rate for both male and female separately. 
3. Find the max and minimum literacy rate for both male and female separately

In [ ]:
!cat mapper#3.py

#!/usr/bin/env python
"""mapper.py"""
import re
import sys
infile=sys.stdin
next(infile)
i=0
# input comes from STDIN (standard input)
for row in infile:
    # remove leading and trailing whitespace
    values = [x.strip() for x in row.split(',')]

    print ('%s\t%s\t%s\t%s\t%s' % (values[1],values[2],values[3],values[4],1))
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
    
        
      

In [ ]:
!cat reducer#3.py

#!/usr/bin/env python
"""reducer.py"""
from heapq import nlargest
from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None
N = 20
word_dict={}
total_female=0
total_male=0
male_LR= []
female_LR= []
total_val=0

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    print('%s' %(line))
    # parse the input we got from mapper.py
    male, female, lm, lf, val = line.split('\t')
    
    total_female = total_female + (int(female)*float(lf))
    total_male = total_male + (int(male)*float(lm))
    male_LR.append(float(lm))
    female_LR.append(float(lf))
    total_val = total_val + int(val)

print('%s\t%s' % ("Total Rows :",(int(total_val))))   
print('%s\t%s' % ("Total Male Literates :",(int(total_male))))
print('%s\t%s' % ("Total Female Literates :",(int(total_female))))
print('%s\t%s' % ("Average Literacy Rate For Male :", sum(male_LR)/len(male_LR)))
print('%s\t%s' % ("Average Literacy R

In [ ]:
!cat Literacy_and_Literacy_Rate.csv | python mapper#3.py | sort -k1,1 | python reducer#3.py

147911	107091	65.99	47.5	1
1542688	1349467	86.99	79.35	1
159704	102751	60.05	39.48	1
168792	112539	65.73	45.27	1
181739	132395	71.21	51.69	1
182946	132665	73.47	52.17	1
210021	151977	69.03	47.14	1
223069	154915	67.51	45.15	1
223575	164672	70.05	50.76	1
228689	165802	73.48	53.4	1
229809	166389	66.52	47.81	1
233711	165359	71.9	50.67	1
247538	162621	64.85	43.64	1
253858	180461	75.82	55.02	1
268587	208774	73.68	57.36	1
269729	202127	72.96	55.59	1
273236	197138	67.58	48.32	1
283542	203017	67.37	46.13	1
307947	229689	70.6	50.38	1
326013	233560	72.3	51.08	1
351411	287288	73.56	59.33	1
357364	276616	77.94	60.38	1
362596	271717	73.39	54.85	1
412868	333592	84.4	67.98	1
429914	302794	66.77	46.83	1
462275	370045	73.69	58.31	1
494705	359255	73.02	54.84	1
504933	395503	74.08	54.95	1
534573	381169	73.9	53.46	1
866798	677161	78.94	64.63	1
961188	816500	87.44	77.34	1
Total Rows :	31
Total Male Literates :	886223787
Total Female Literates :	541228067
Average Literacy Rate For Male :	72.39419354838711
Av

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!cat US_AirlLines.txt | python mapper#2.py | sort -k1,1 | python reducer#2.py

In [ ]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz

--2021-11-16 07:20:56--  https://downloads.apache.org/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 500749234 (478M) [application/x-gzip]
Saving to: ‘hadoop-3.3.0.tar.gz’

hadoop-3.3.0.tar.gz 100%[===================>] 477.55M  23.4MB/s    in 23s     

2021-11-16 07:21:19 (21.0 MB/s) - ‘hadoop-3.3.0.tar.gz’ saved [500749234/500749234]



In [ ]:
!tar -xzvf hadoop-3.3.0.tar.gz

Streaming output truncated to the last 5000 lines.
hadoop-3.3.0/share/doc/hadoop/hadoop-project-dist/hadoop-common/build/source/hadoop-common-project/hadoop-common/target/api/org/apache/hadoop/fs/FSDataOutputStream.html
hadoop-3.3.0/share/doc/hadoop/hadoop-project-dist/hadoop-common/build/source/hadoop-common-project/hadoop-common/target/api/org/apache/hadoop/fs/TrashPolicyDefault.Emptier.html
hadoop-3.3.0/share/doc/hadoop/hadoop-project-dist/hadoop-common/build/source/hadoop-common-project/hadoop-common/target/api/org/apache/hadoop/fs/HarFileSystem.html
hadoop-3.3.0/share/doc/hadoop/hadoop-project-dist/hadoop-common/build/source/hadoop-common-project/hadoop-common/target/api/org/apache/hadoop/fs/PathExistsException.html
hadoop-3.3.0/share/doc/hadoop/hadoop-project-dist/hadoop-common/build/source/hadoop-common-project/hadoop-common/target/api/org/apache/hadoop/fs/XAttrSetFlag.html
hadoop-3.3.0/share/doc/hadoop/hadoop-project-dist/hadoop-common/build/source/hadoop-common-project/hadoop-

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
#To find the default Java path
!readlink -f /usr/bin/java | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


In [ ]:
#copy  hadoop file to user/local
!cp -r hadoop-3.3.0/ /usr/local/

In [ ]:
#Running Hadoop
!/usr/local/hadoop-3.3.0/bin/hadoop

Usage: hadoop [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]
 or    hadoop [OPTIONS] CLASSNAME [CLASSNAME OPTIONS]
  where CLASSNAME is a user-provided Java class

  OPTIONS is none or any of:

buildpaths                       attempt to add class files from build tree
--config dir                     Hadoop config directory
--debug                          turn on shell script debug mode
--help                           usage information
hostnames list[,of,host,names]   hosts to use in slave mode
hosts filename                   list of hosts to use in slave mode
loglevel level                   set the log4j level for this command
workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

daemonlog     get/set the log level for each daemon

    Client Commands:

archive       create a Hadoop archive
checknative   check native Hadoop and compression libraries availability
classpath     prints the class path needed to get the Hadoop jar and the
    

In [ ]:
!mkdir ~/input
!cp /usr/local/hadoop-3.3.0/etc/hadoop/*.xml ~/input

In [ ]:
!ls ~/input

capacity-scheduler.xml	hdfs-rbf-site.xml  kms-acls.xml     yarn-site.xml
core-site.xml		hdfs-site.xml	   kms-site.xml
hadoop-policy.xml	httpfs-site.xml    mapred-site.xml


In [ ]:
!wget http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz

!tar -xzvf 20news-18828.tar.gz

Streaming output truncated to the last 5000 lines.
20news-18828/sci.space/61079
20news-18828/sci.space/60809
20news-18828/sci.space/61497
20news-18828/sci.space/60156
20news-18828/sci.space/61475
20news-18828/sci.space/59873
20news-18828/sci.space/61541
20news-18828/sci.space/61489
20news-18828/sci.space/61133
20news-18828/sci.space/62418
20news-18828/sci.space/61289
20news-18828/sci.space/60864
20news-18828/sci.space/61443
20news-18828/sci.space/60189
20news-18828/sci.space/60251
20news-18828/sci.space/61356
20news-18828/sci.space/61255
20news-18828/sci.space/61345
20news-18828/sci.space/61382
20news-18828/sci.space/60847
20news-18828/sci.space/62481
20news-18828/sci.space/60783
20news-18828/sci.space/60854
20news-18828/sci.space/62131
20news-18828/sci.space/60921
20news-18828/sci.space/62317
20news-18828/sci.space/61372
20news-18828/sci.space/61406
20news-18828/sci.space/60247
20news-18828/sci.space/62114
20news-18828/sci.space/60791
20news-18828/sci.space/61362
20news-18828/sci.spac

In [ ]:
!find / -name 'hadoop-streaming*.jar'

/usr/local/hadoop-3.3.0/share/hadoop/tools/sources/hadoop-streaming-3.3.0-sources.jar
/usr/local/hadoop-3.3.0/share/hadoop/tools/sources/hadoop-streaming-3.3.0-test-sources.jar
/usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
/content/hadoop-3.3.0/share/hadoop/tools/sources/hadoop-streaming-3.3.0-sources.jar
/content/hadoop-3.3.0/share/hadoop/tools/sources/hadoop-streaming-3.3.0-test-sources.jar
/content/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar


In [ ]:
!chmod u+rwx /content/mapper.py
!chmod u+rwx /content/reducer.py

In [ ]:
!/usr/local/hadoop-3.3.0/bin/hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input /content/20news-18828/alt.atheism/49960 -output /content/output -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'

2021-11-16 07:50:35,704 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/content/mapper.py, /content/reducer.py] [] /tmp/streamjob5609518188227250556.jar tmpDir=null
2021-11-16 07:50:36,378 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2021-11-16 07:50:36,471 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2021-11-16 07:50:36,471 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2021-11-16 07:50:36,487 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2021-11-16 07:50:36,634 INFO mapred.FileInputFormat: Total input files to process : 1
2021-11-16 07:50:36,656 INFO mapreduce.JobSubmitter: number of splits:1
2021-11-16 07:50:36,865 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local312102315_0001
2021-11-16 07:50:36,866 INFO mapreduce.JobSubmitter: Executing with tokens: []
2021-11-16 07:50:37,198 INFO mapred.Loca

In [ ]:
!ls /content/output

part-00000  _SUCCESS


In [ ]:
!cat /content/output/part-00000

034529887x	1
0511211216	1
071	5
080182494x	1
0801834074	1
0877226423	1
0877227675	1
0908	1
0910309264	1
1	1
10	1
11	1
1266	1
1271	1
14	1
140195	1
14215	1
14226	1
142282197	1
17701900	1
1881	1
1977	1
1981	1
1986	1
1988	1
1989	1
1990	1
1992	1
20th	1
226	1
24hour	1
2568900	1
272	1
273	1
2nd	1
3005	1
316	1
372	1
3d	1
3nl	1
4	1
41	2
430	2
4581244	1
4679525	1
490	1
495	2
4rh	1
4rl	1
512	2
53701	1
541	1
59	1
608	1
664	1
700	1
702	1
7119	1
716	1
7215	1
7251	1
750	1
7723	1
787140195	1
787522973	1
831	1
8372475	1
88	1
880	2
8964079	1
8ew	1
91605	1
aah	1
aap	2
abortions	1
absurdities	1
accompanied	1
accounts	1
address	1
addresses	2
adulteries	1
aesthetics	1
african	3
africanamericans	1
agnostic	1
al	1
alien	1
allen	2
also	3
altatheism	1
altatheismarchivename	1
altatheismmoderated	1
alternate	1
alternative	1
although	2
america	1
american	5
americans	2
amherst	1
amongst	1
amusing	1
ancient	1
andor	1
another	1
anselm	1
anthology	3
anyone	1
appendix	2
approachable	1
archive	1
archivename	1
archives	1

In [ ]:
!cat RandomText.txt | python mapper.py | sort -k1,1 | python reducer.py

allied	1
arrow	1
behind	1
believing	1
came	1
capture	1
chest	1
commanding	1
cover	1
cried	1
dying	1
eventually	1
farmhouse	1
feathered	1
fire	2
forces	1
german	2
germans	1
glanced	1
heavy	1
located	1
long	1
low	1
managed	1
manâs	1
manchester	1
nazi	1
nazis	1
officer	1
one	1
pin	1
protruding	1
regiment	1
see	1
shot	1
soldier	1
soldiers	2
sought	1
squadron	1
strategically	1
suppressing	1
took	1
village	1
wall	1


In [ ]:
!cp Literacy_and_Literacy_Rate.csv RandomText.txt US_AirlLines.txt cano.txt mapper.py reducer.py  mapper#2.py mapper#3.py reducer#2.py reducer#3.py /content/drive/MyDrive/MapReduce